In [183]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [184]:

# 데이터 로드 및 전처리
digits = load_digits()


In [185]:
digits

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [186]:
X = digits.data.astype(np.float32)
y = digits.target.astype(np.int64)

In [187]:
X

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]], dtype=float32)

In [188]:
y

array([0, 1, 2, ..., 8, 9, 8], dtype=int64)

In [189]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15/0.85, random_state=42)

In [190]:

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)  # 분류용 레이블은 int64(LongTensor)로
X_test  = torch.tensor(X_test,  dtype=torch.float32)
y_test  = torch.tensor(y_test,  dtype=torch.int64)
X_val   = torch.tensor(X_val,   dtype=torch.float32)
y_val   = torch.tensor(y_val,   dtype=torch.int64)

In [191]:
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

(torch.Size([1257, 64]),
 torch.Size([270, 64]),
 torch.Size([270, 64]),
 torch.Size([1257]),
 torch.Size([270]),
 torch.Size([270]))

In [192]:
# DataLoader 정의 (배치 크기 32)
train_loader = DataLoader(
    TensorDataset(X_train, y_train),
    batch_size=32, shuffle=True
)
val_loader = DataLoader(
    TensorDataset(X_val, y_val),
    batch_size=32, shuffle=False
)
test_loader = DataLoader(
    TensorDataset(X_test, y_test),
    batch_size=32, shuffle=False
)

In [193]:


class DigitDNN(nn.Module):
    def __init__(self):
        super(DigitDNN, self).__init__()
        self.fc1 = nn.Linear(64, 80)
        self.fc2 = nn.Linear(80, 80)
        self.fc3 = nn.Linear(80, 64)
        self.fc4 = nn.Linear(64, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DigitDNN()


# 손실함수 및 최적화 기법 설정
criterion = nn.CrossEntropyLoss()         # 다중 클래스 분류용
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [194]:


# 4) 학습 루프
num_epochs = 30
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()                    # 기울기 초기화
        outputs = model(X_batch)                 # 순전파
        loss = criterion(outputs, y_batch)       # 손실 계산
        loss.backward()                          # 역전파
        optimizer.step()                         # 가중치 갱신
        running_loss += loss.item() * X_batch.size(0)

    avg_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch:2d}/{num_epochs} — Loss: {avg_loss:.4f}")

# 5) 평가
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, preds = torch.max(outputs, dim=1)    # 가장 높은 확률 클래스 선택
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# 성능 지표 출력
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))


Epoch  1/30 — Loss: 1.5614
Epoch  2/30 — Loss: 0.3829
Epoch  3/30 — Loss: 0.1963
Epoch  4/30 — Loss: 0.1430
Epoch  5/30 — Loss: 0.0977
Epoch  6/30 — Loss: 0.0834
Epoch  7/30 — Loss: 0.0571
Epoch  8/30 — Loss: 0.0570
Epoch  9/30 — Loss: 0.0408
Epoch 10/30 — Loss: 0.0303
Epoch 11/30 — Loss: 0.0291
Epoch 12/30 — Loss: 0.0243
Epoch 13/30 — Loss: 0.0394
Epoch 14/30 — Loss: 0.0330
Epoch 15/30 — Loss: 0.0279
Epoch 16/30 — Loss: 0.0101
Epoch 17/30 — Loss: 0.0068
Epoch 18/30 — Loss: 0.0070
Epoch 19/30 — Loss: 0.0066
Epoch 20/30 — Loss: 0.0033
Epoch 21/30 — Loss: 0.0027
Epoch 22/30 — Loss: 0.0028
Epoch 23/30 — Loss: 0.0025
Epoch 24/30 — Loss: 0.0020
Epoch 25/30 — Loss: 0.0016
Epoch 26/30 — Loss: 0.0015
Epoch 27/30 — Loss: 0.0014
Epoch 28/30 — Loss: 0.0013
Epoch 29/30 — Loss: 0.0012
Epoch 30/30 — Loss: 0.0011

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       0.95      1.00      0.97        18
   

In [195]:

# 5) 평가
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, preds = torch.max(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# 성능 지표 출력
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))



Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       0.95      1.00      0.97        18
           2       0.96      1.00      0.98        26
           3       0.96      0.96      0.96        24
           4       1.00      1.00      1.00        37
           5       0.89      0.97      0.93        33
           6       1.00      0.97      0.98        30
           7       1.00      0.96      0.98        26
           8       0.94      0.89      0.92        19
           9       1.00      0.94      0.97        34

    accuracy                           0.97       270
   macro avg       0.97      0.97      0.97       270
weighted avg       0.97      0.97      0.97       270

Confusion Matrix:
[[23  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 26  0  0  0  0  0  0  0]
 [ 0  0  0 23  0  1  0  0  0  0]
 [ 0  0  0  0 37  0  0  0  0  0]
 [ 0  0  1  0  0 32  0  0  0  0]
 